# FINAL PLANNER DEMOSTRATION

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from algorithm.planner import JourneyPlanner
from collections import defaultdict
import pickle
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import plotly
import matplotlib.pyplot as plt
import pickle
from algorithm.journey import *

In [3]:
with open('../data/footpath.pickle', 'rb') as f:
    footpaths = pickle.load(f)

In [4]:
with open('../data/trips.pickle', 'rb') as f:
    trips = pickle.load(f)

In [5]:
with open('../data/stops.pickle', 'rb') as f:
    stops = pickle.load(f)

In [6]:
with open('../data/connections_data.pickle', 'rb') as f:
    connections = pickle.load(f)

In [7]:
with open('../data/confidence.pickle', 'rb') as f:
    confidences = pickle.load(f)

In [8]:
with open('../data/routes.pickle', 'rb') as f:
    routes = pickle.load(f)

In [9]:
timetable = stops, connections, trips, footpaths, confidences

In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [11]:
stop_names = []
for key in stops.keys():
    stop_names.append(stops[key]["stop_name"])

stop_names = np.unique(sorted(stop_names))        #getting all the stationnames and sorting them in alphabetical order

In [12]:
# Below we define all the widgets
import datetime
import warnings
from visualization.visualization_helpers import visualize_map_scattermapbox

warnings.simplefilter(action='ignore', category=FutureWarning)
confidence_level =widgets.FloatSlider(
    description = "Confidence Interval",
    value=0,
    min=0,
    max=1.0,
    step=0.01,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style= {'description_width': 'initial'}
)

datetime_picker = widgets.DatePicker(
    value = datetime.datetime.now().date(),
    description='Date of the trip',
    disabled=False,
    style= {'description_width': 'initial'}
)

aux = datetime.datetime.now()
max_arrival_time_hour = widgets.Dropdown(
    value = datetime.datetime.now().time().hour + 3 if datetime.datetime.now().time().hour + 1 in range(10,16) else 13 ,
    options= [i for i in range(9,18)],
    description='Max Arrival Hour:',
    disabled=False,
    style= {'description_width': 'initial'}
)

max_arrival_time_minute = widgets.Dropdown(
    value = datetime.datetime.now().time().minute,
    options= [i for i in range(1,60)],
    description='Max Arrival Minute:',
    disabled=False,
    style= {'description_width': 'initial'}
)


min_departure_time_hour = widgets.Dropdown(
    value = datetime.datetime.now().time().hour if datetime.datetime.now().time().hour in range(9,15) else 11 ,
    options= [i for i in range(9,18)],
    description='Min Departure Hour:',
    disabled=False,
    style= {'description_width': 'initial'}
)


min_departure_time_minute = widgets.Dropdown(
    value = datetime.datetime.now().time().minute,
    options= [i for i in range(1,60)],
    description='Min Departure Minute:',
    disabled=False,
    style= {'description_width': 'initial'}
)



departure_station = widgets.Dropdown(
    options= stop_names,
    description='Departure Station:',
    disabled=False,
    style= {'description_width': 'initial'}
)

arrival_station = widgets.Dropdown(
    options= stop_names,
    description='Arrival Station:',
    disabled=False,
    style= {'description_width': 'initial'}
)


max_changes = widgets.IntText(
    value=10,
    description='Max. number of changes:',
    disabled=False,
)



show_k_journeys = widgets.Dropdown(
    value = 5,
    options= [i for i in range(1,11)],
    description='Max n. of journeys to display:',
    disabled=False,
    style= {'description_width': 'initial'}
)

layout = widgets.Layout(width='auto', height='40px')

button = widgets.Button(
    description='Find the best route!',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    layout = layout
)


out = widgets.Output()

# The following functions defines what happens when we press the button

                             
def button_action(button):
    
    out.clear_output()   # each time we press the button we clear the previous output
    
    with out:

        dep_station_name = departure_station.value
        arr_station_name = arrival_station.value
        if dep_station_name == arr_station_name:
            print("You cannot choose the same station")
            return
        
        dep_station_ids = [k for k,v in stops.items() if v['stop_name'] == dep_station_name] #exctracting all ids related to the departure station name. Later, we use only the first one (i.e. as input of the planner we only give dep_station_ids[0])
        arr_station_ids = [k for k,v in stops.items() if v['stop_name'] == arr_station_name] #exctracting all ids related to arrival station name. Later, we use only the first one (i.e. as input of the planner we only give arr_station_ids[0])
        
            
        weekday = datetime_picker.value.weekday()
        if not(weekday in [0,1,2,3,4]):
            print("You cannot choose Saturday or Sunday")
            return

        max_time = datetime.time(max_arrival_time_hour.value, max_arrival_time_minute.value)
        min_time = datetime.time(min_departure_time_hour.value, min_departure_time_minute.value)
        if max_time <= min_time:
            print("You cannot start after terminating the journey")
            return
            
        date_value_from_connections = datetime.datetime.fromtimestamp(connections[0][2]) #extracting the date timestamp from connections, since the algo needs this day in order to run

        timestamp1 = int(datetime.datetime.combine(date_value_from_connections, min_time).timestamp()) #combining the date from connections with the min time provided by the user
        timestamp2 = int(datetime.datetime.combine(date_value_from_connections, max_time).timestamp()) #combining the date from connections with the max time provided by the user
        inputs = [weekday, dep_station_ids[0], arr_station_ids[0], timestamp1, timestamp2, max_changes.value, confidence_level.value]

        planner = JourneyPlanner(timetable)
        print("Loading...")
        journeys = planner.plan_route(weekday, dep_station_ids[0], arr_station_ids[0], timestamp1, timestamp2, confidence_level.value, max_changes.value, verbose=True)
        

        print("Done. Total Number of journeys found", len(journeys))
        max_journeys = show_k_journeys.value #used to visualize only first k journeys
            
        for i in range(len(journeys[:max_journeys])): # we plot only first k journeys
            visualize_map_scattermapbox(journeys[i], routes, i+1) 
        
button.on_click(button_action)

In [13]:
import datetime
display(datetime_picker, min_departure_time_hour,min_departure_time_minute, max_arrival_time_hour, max_arrival_time_minute , departure_station, arrival_station, confidence_level, max_changes, show_k_journeys, button, out)

DatePicker(value=datetime.date(2022, 5, 29), description='Date of the trip', style=DescriptionStyle(descriptio…

Dropdown(description='Min Departure Hour:', index=2, options=(9, 10, 11, 12, 13, 14, 15, 16, 17), style=Descri…

Dropdown(description='Min Departure Minute:', index=48, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

Dropdown(description='Max Arrival Hour:', index=4, options=(9, 10, 11, 12, 13, 14, 15, 16, 17), style=Descript…

Dropdown(description='Max Arrival Minute:', index=48, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …

...

FloatSlider(value=0.0, continuous_update=False, description='Confidence Interval', max=1.0, step=0.01, style=S…

IntText(value=10, description='Max. number of changes:')

Dropdown(description='Max n. of journeys to display:', index=4, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), style…

Button(description='Find the best route!', icon='check', layout=Layout(height='40px', width='auto'), style=But…

Output()